In [1]:
from LFOFM import *
import threading
import OSC

wp = WavePlayer()
tempo = wp.setTempo

import PydalChanel as pydalModule

read = pydalModule.read
pydal = pydalModule.getPydalInstance()
port = pydal.port
ch1 = pydal.newChannel(1)
ch2 = pydal.newChannel(2)

triggerFunctions = {}

def sendChord(chord):
    scClient = pydal.superColliderClient
    msg = OSC.OSCMessage()
    msg.setAddress("/waveNotes")
    msg.append(chord)
    scClient.send(msg)
    

def funcTriggerResponder(addr, tags, stuff, source):
    if stuff[0] in triggerFunctions:
        triggerFunctions[stuff[0]]()
        #could have the "drumStr" be something like "a_param1_param2_..._" and feed string to function,
        #splitting off the first "_" for the function id

pydal.superColliderServer.addMsgHandler("/funcTrigger", funcTriggerResponder)

In [4]:
bf = (Sin(0.01)+1)*1

q = 10 + (Sin(freq=Sin(0.08)+2)+1.)/2. * 30

f1 = (Sin(freq=0.5, phase=Tri(1.5 * bf))*12 + 64) 
m1 =Sin(0.2)*(Sqr() + 1.)*12.
q1 = 0.5 + (Sin(0.1)+1.)/2. * 2

f2 = (Sin(freq=0.03, phase=Sin(1*bf))*8 + 40)
m2 = Saw(0.1)*-12
q2 = 1 + (Sin(0.1, 0.7)+1.)/2. * 10

f3 = (Sin(phase=Sin(2  * bf))*6 + 50)
q3 = 1 + (Sin(0.1, 1.4)+1.)/2. * 10

waves = [[f3+m1, q], [f1+m1, q], [f2+m2, q2], [f3, q3], [f2+m1, q1], [f1+m2, q], [f3+m1, q3]]

In [7]:
from LFOFM import *
from treeBuilder import TreeBuilder
import random

class Counter:
    def __init__(self):
        self.count = 0
    def __call__(self):
        count = self.count
        self.count += 1
        return self.count 


waveTrees = [0, 0, 0]
for i in range(len(waveTrees)):
    c = Counter()
    waveTrees[i] = TreeBuilder(waves[0], lambda a: waves[c()], fullTreeDepth=2)

c = Counter()
chords = [random.sample(range(12), 3) for i in range(8)]
chordTree = TreeBuilder(chords[0], lambda a: chords[c()], fullTreeDepth=2)
    

def randWalk():
    return random.choice(["_"]*3 + ["\/:0", "\/:1", ">", "<", "^", "^"])


def wavesWalk():
    for i in range(len(waveTrees)):
        waveTree = waveTrees[i]
        waveTree.execute(randWalk())
        waves = waveTree.currentNode.value
        wp.startWaveOSC('f'+str(i+1), '/'+str(i+1)+'/freq', waves[0], metaInfo="warp")
        wp.startWaveOSC('q'+str(i+1), '/'+str(i+1)+'/q', waves[1])

def chordWalk():
    chordTree.execute(randWalk())
    sendChord(chordTree.currentNode.value)
                
def fun2():
    sendChord([3,7,11])
triggerFunctions["a"] = wavesWalk
triggerFunctions["b"] = chordWalk

timeSteps = [2.0, 2.0, 4.0, 4.0, 8.0] 
random.shuffle(timeSteps)
chordsRhythm = [sum(timeSteps[:i+1]) for i in range(len(timeSteps))]
random.shuffle(timeSteps)
wavesRhythm = [sum(timeSteps[:i+1]) for i in range(len(timeSteps))]


wavesPat = pydalModule.FlatGenericPattern([[0.0, "a"]], "funcTrigger", lambda a:a)
wavesPat.frac = sum(timeSteps)
chordsPat = pydalModule.FlatGenericPattern([[t, "b"] for t in chordsRhythm], "funcTrigger", lambda a:a)
chordsPat.frac = sum(timeSteps)
ch1.play(wavesPat, metaInfo=[port])
ch2.play(chordsPat, metaInfo=[port])

In [ ]:
ch1.stop()
ch2.stop()
# ch1.play(wavesPat, metaInfo=[port])
# ch2.play(chordsPat, metaInfo=[port])